# TTV Project

Identification des génotypes de Torque teno virus "TTV" présents chez les donneurs "D" et receveurs "R" de greffe. <br>

## Summary

1. Introduction <br>
2. Goals <br>
3. Data <br>
3.1. Raw reads <br>
3.2. Human genome <br>
3.3. TTV genotypes <br>
3.4. Microbial databases <br>
3.4.1. Viral <br>
3.4.2. Fungal <br>
3.4.3. Bacterial <br>
4. Reads filtering <br>
4.1. Initial quality control <br>
4.2. Filtering based on read quality <br>
4.2.1. QC <br>
4.2.2. Merge FASTQ files belonging to the same sample <br>
4.3. Human reads removal <br>
4.3.1. Load data on genotoul <br>
4.3.2. Build BWA index <br>
4.3.3. Map reads <br>
4.3.4. Validate BAMs <br>
4.3.5. Sort & index BAMs <br>
4.3.6. BAM to FASTA <br>
4.3.7. DL these FASTA files <br>
4.4. 5' UTR reads removal <br>
5. TTV taxonomical analysis <br>
5.1. Retrieve Orf1 sequences for our 57 genotypes <br>
5.2. Create taxo groups based on Orf1 homology <br>
6. Assign reads to microbial DBs & TTV genotypes <br>
6.1. Create databases <br>
6.1.1. Viral DBs <br>
6.1.2. Fungal DB <br>
6.1.3. Bacterial DB <br>
6.2. Blastn vs viral DBs <br>
6.3. Blastn vs fungal DB <br>
6.4. Blastn vs bacterial DB <br>
6.5. Hit count per TTV genotypes <br>

## 0) Todo

- Terminer le projet pour fin Septembre. <br>
- Check DB MIC (contient tt génomes micro-org ?). <br>
- [Valentin] Contacter Rémi et voir où il en était sur son script de classification viral. <br>
- Lancer le pipeline d'analyse complet ASAP (+ réflection sur étape blastn, need pos hit pour éventuel sort -> Orf1 vs complet) (et trouver un "mode BAM" pour hits blastn pour faire pileup easily). <br>

## 1) Introduction <a name="introduction_1"></a>
Cf rapport sous overleaf : https://www.overleaf.com/10502404cfvdmgkmjwtc <br>

## 2) Goals

X) L'analyse comportera 2 "embranchements" : <br>
- Chemin Orf1 || génomes_complets
- Puis raisonnement sur génotypes pris 1 à 1 || groupes phylo de génotypes. <br>
Ps : Pour voir si on s'attend à une grande différence entre Orf1 & complet, plot par génotypes le pileup des reads (des hits) sur séquences génome complet de l'étude. <br>

A) Décrire les génotypes de TTV présents chez les D & R (D_all, R_all, R_35, R_75, R_360). <br>

B) Des génotypes de TTV sont-ils transmis de D vers R, si oui quels sont-ils ? Certains ne sont-ils jamais transmis ? Taux de transmission ? Taux / succès de transmission est-il lié à la charge virale du D ? <br>

C) Quels sont les génotypes qui croissent ou décroissent chez les R (quel(s) G(s) explique(nt) la plus grande partie de l'augmentation de la charge virale) ? Viennent-ils de D ? Sont-ce les mêmes 2/3 génotypes / groupe phylo de génotypes qui croissent chez tout les R ? (doit-on rapporter à la charge virale ?) <br>

D) Les génotypes trouvés chez D & R sont-ils les mêmes d'un point de vue séquence ? Ex : D transmet G18 à R. G18 subit-il des modifs chez R ? (need assembly or something here => tool publi gigascience ?). Ps : Check aussi si recombinaison entre Gx & Gy (check via reads chevauchants ?). Un point important ici va être de donner la bonne sélection de reads en entrée d'un éventuel assemblage (donner filtered -Bact -Fungus ? Quid de refiltrer sur les no-match?). Et selon ce qu'on arrive à assembler, il faut prendre à peu près la même séquence pour comparer ensuite (genre si on assemble 3 G complet et 7 Orf1, prendre 10 Orf1 pour les comparaisons). <br>

Exploratoire) "Top 10" des familles / espèces / ... les plus fréquentes lors du blastn vs Viral, Fungal & Bacterial. Pour max X reads qui ne se sont alignés contre rien, blast vs nt / nr.

## 3) Data

### 3.1) Raw reads

106 FASTA files coming from 3 differents Ion Proton runs (GénoLim). <br>

Number of reads in raw files :

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Data/Reads_raw;
for f in *fastq.gz;
do lines=$(zgrep -Ec "$" $f);
   printf $(($lines/4));
   printf '\t'$f'\n';
done;

Data to sample correspondance : <br>

Ps : The tidy datasheet below was deduced from : <br>
~/Desktop/TTV/BISCEm/Docs/corresp_D-R_barecode_16062017.ppt <br>
~/Desktop/TTV/BISCEm/Docs/donneurs_receveurs.xlsx.

In [ ]:
#### Code ###
chromium-browser https://docs.google.com/spreadsheets/d/\
1KI94IUmesq1CZwdfSobA2qzEM8ZeqHWh8sR3QPar_K0/edit?usp=sharing;

### 3.2) Human genome

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Data/Human;
axel ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/\
GCA_000001405.26_GRCh38.p11/GCA_000001405.26_GRCh38.p11_genomic.fna.gz;
md5sum GCA_000001405.26_GRCh38.p11_genomic.fna.gz; # f48d3166ede36c3381643dace60348b9

### 3.3) TTV genotypes

Retrieval sources :

- NCBI taxonomy (29) : ID = 687331 <=> Alphatorquevirus : <br>
Don't take those listed as "simian". <br>
Manually retrieve them, then cat into > ~/Desktop/TTV/BISCEm/Data/TTV/ncbi_taxid_687331_ttv_20170622.fasta <br>


- Reference genome (1) : <br>
https://www.ncbi.nlm.nih.gov/nuccore/NC_015783.1 > ~/Desktop/TTV/BISCEm/Data/TTV/ncbi_ttv_ref_genome_20170622.fasta.gz <br>


- Via entrez search terms (27) : <br>
https://ncbiinsights.ncbi.nlm.nih.gov/2013/02/19/how-to-download-bacterial-genomes-using-the-entrez-api <br> https://www.ncbi.nlm.nih.gov/books/NBK179288

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Data/TTV;

# Retrieve TTV ID (!= taxo)
wget -O ttv_genome.xml "https://eutils.ncbi.nlm.nih.gov/entrez/\
eutils/esearch.fcgi?db=genome&term=torque+teno+virus";
id=$(grep -oP '<Id>\K.*?(?=<\/Id>)' ttv_genome.xml);

# Use this ID to get all TTV genomes IDs from nuccore database
wget -O ttv_genomes_ids.xml "https://eutils.ncbi.nlm.nih.gov/entrez\
/eutils/elink.fcgi?dbfrom=genome&db=nuccore&id=$id";
grep -oP '<Id>\K.*?(?=<\/Id>)' ttv_genomes_ids.xml > ttv_genomes_ids.txt;
sed -i '1d' ttv_genomes_ids.txt;

# Filter out genomes IDs not associated with H.sapiens, then retrieve associated FASTA sequences
while read line;
do wget -O temp.txt "https://eutils.ncbi.nlm.nih.gov/entrez/\
eutils/efetch.fcgi?db=nuccore&id=$line&retmode=text"; 
   if grep --quiet "sapiens" temp.txt;
   then echo $line >> ttv_genomes_ids_with_hsapiens.txt;
   fi;
done < ttv_genomes_ids.txt;
while read line;
do wget -O - "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/\
efetch.fcgi?db=nuccore&id=$line&rettype=fasta&retmode=text" >> entrez_ttv_genomes_with_hsapiens_20170622.fasta;
done < ttv_genomes_ids_with_hsapiens.txt;
pigz entrez_ttv_genomes_with_hsapiens_20170622.fasta;

# Clean all these temp files mess
rm ttv_genome.xml ttv_genomes_ids.xml ttv_genomes_ids.txt temp.txt ttv_genomes_ids_with_hsapiens.txt;

Then cat those 3 files in a single FASTA file (containing the 57 genotypes) :

In [ ]:
#### Code ###
cat ncbi_taxid_687331_ttv_20170622.fasta.gz \
    ncbi_ttv_ref_genome_20170622.fasta.gz \
    entrez_ttv_genomes_with_hsapiens_20170622.fasta.gz > all_ttv_genotypes.fasta.gz;

### 3.4) Microbial databases

#### 3.4.1) Viral

#### 3.4.2) Fungal

#### 3.4.3) Bacterial

Given that the NCBI refseq contain too much sequences for the future blastn step to be done in a reasonable amount of computing time, we needed to reduce the size of the bacterial database. The choice was made to use The Human microbiome project (HMP) as a ressource. <br>
In 2009, HMP compiled a database of microbial reference genomes (HMREFG), as described here : http://www.hmpdacc.org/HMREFG. This database contains archea, low-eucaryote, viral & bacterial, which are reported in the fasta header as follow : >ARCH, >EUKY, >VIRL, >BACT. The bacterial component of this database underwent a process of removing highly redundant, non HMP-sequenced reference genomes.

Retrieve the HMREFG file, decompress & rename it :

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Data/Bacterial;
axel -q ftp://public-ftp.hmpdacc.org/HMREFG/all_seqs.fa.bz2;
bzip2 -d all_seqs.fa.bz2;
mv all_seqs.fa hmp_microorganisms.fasta;

Check which bacteria are included in this database :

In [ ]:
#### Code ###
grep '^>BAC' hmp_microorganisms.fasta \
| awk -F "[[:space:]]" '{print $2"_"$3}' \
| sort -u > hmp_bacterial_list.txt;
head hmp_bacterial_list.txt;
tail hmp_bacterial_list.txt;

Retrieve each bacteria header, then use seqtk to subsample all bacteria in the hmp_microorganisms.fasta :

In [ ]:
#### Code ###
grep '^>BAC' hmp_microorganisms.fasta \
| awk -F "[[:space:]]" '{print $1}' \
| sed 's/^>//g' > hmp_bacterial_header.list;
seqtk subseq hmp_microorganisms.fasta hmp_bacterial_header.list > hmp_bacterial.fasta;

Compress data for storage and remove useless files :

In [ ]:
#### Code ###
pigz hmp_bacterial.fasta;
pigz hmp_microorganisms.fasta;
rm hmp_bacterial_header.list;

Ps : Bonus question : The HMREFG Reference Genome Database contains (quoting) "1751 bacterial strains over 1253 species". Let's try to verify this :

In [ ]:
#### Code ###
zgrep '^>BAC' hmp_microorganisms.fasta.gz \
| awk -F '\\|' '{print $1}' \
| awk -F "_" '{print $2}' \
| sort -un \
| wc -l;

We are 17 strains short. This question was mailed to http://www.hmpdacc.org/outreach/feedback.php.

Ps bis : Get bacteria accession numbers :

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Data/Bacterial;
awk -F "[[:space:]]" '{print $1}' hmp_bacterial_header.list | \
rev | sed 's/^|//' | cut -d'|' -f 1 | rev > hmp_bacterial_an.txt;

## 4) Reads filtering

### 4.1) Initial quality control

FastQC v0.11.5 :

In [ ]:
#### Code ###
mkdir -p ~/Software/FastQC/Outputs/TTV ~/Desktop/TTV/BISCEm/Data/Reads_raw/QC;
cd ~/Desktop/TTV/BISCEm/Data/Reads_raw;

In [ ]:
#### Code ###
for f in *.fastq.gz;
do fastqc --outdir ~/Software/FastQC/Outputs/TTV/ -f fastq $f;
done >> ~/Desktop/TTV/BISCEm/Data/Reads_raw/QC/fastqc_verbose.txt;

multiqc, version 0.9.dev0 : <br>
Ps : Done for all combined runs & for each run separately, to highlight potential run biais.

In [ ]:
#### Code ###
# For all runs
cd ~/Software/MultiQC/Outputs;
multiqc -i ttv -n ttv -v ~/Software/FastQC/Outputs/TTV/ >> multiqc_verbose.txt;
cp -r ttv* ~/Desktop/TTV/BISCEm/Data/Reads_raw/QC;

# Per run
cd ~/Software/FastQC/Outputs/TTV;
mkdir -p P1 P2 P31 P32;
for f in p1_*; do mv $f P1; done;
for f in p2_*; do mv $f P2; done;
for f in p31_*; do mv $f P31; done;
for f in p32_*; do mv $f P32; done;
cd ~/Software/MultiQC/Outputs;
multiqc -i ttv_p1 -n ttv_p1 -v ~/Software/FastQC/Outputs/TTV/P1;
multiqc -i ttv_p2 -n ttv_p2 -v ~/Software/FastQC/Outputs/TTV/P2;
multiqc -i ttv_p31 -n ttv_p31 -v ~/Software/FastQC/Outputs/TTV/P31;
multiqc -i ttv_p32 -n ttv_p32 -v ~/Software/FastQC/Outputs/TTV/P32;
cp -r *ttv_p* ~/Desktop/TTV/BISCEm/Data/Reads_raw/QC;
cd ~/Software/FastQC/Outputs/TTV;
mv P1/* P2/* P31/* P32/* .;
rm -r P1 P2 P31 P32;

Visualize multiqc results :

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Data/Reads_raw/QC;
chromium-browser *.html;

Comments : <br>
- The 5' per base sequence content seems skewed on the first 10 bases (i.e. not 25% for each nt), but this is expected for libraries produced by priming using random hexamers, which induce sequence composition biais on 5'. <br>
- The over-represented sequence metric has no real sense for datasets with very few (100000 or less) reads. <br>
- The per base sequence content is dirty toward the 3' end for our datasets, because there are a few (<1%%) sequences which are 200+ nts long, thus the ATGC% is obviously skewed.

### 4.2) Filtering based on read quality

Software : PRINSEQ-lite 0.20.4

In [ ]:
#### Code ###
biscem='/home/erwann/Desktop/TTV/BISCEm';
cd $biscem/Data/Reads_raw;
mkdir -p $biscem/Output/Reads_qual;
out=$biscem'/Output/Reads_qual';

In [ ]:
### CODE ###
echo $PRIN;

In [ ]:
#### Code ###
for f in *.fastq.gz;
do echo $f;
   unpigz $f;
   perl $PRIN -fastq ${f%.*} \
              -out_good $out'/'${f%%.*}'_qual' \
              -out_bad null \
              -trim_qual_type mean \
              -trim_qual_window 5 \
              -trim_qual_left 20 \
              -trim_qual_right 20 \
              -min_len 100 \
              -min_qual_mean 20 \
              -lc_method entropy \
              -lc_threshold 70 \
              -derep 1 \
              -derep_min 2;
   pigz ${f%.*};
   pigz  $out'/'${f%%.*}"_qual".fastq;
done &>> $out'/prinseq.verbose';

Order in which options are applied (see man prinseq) & what they do : <br>

Ps : Triming options : A 5nts sliding windows checking mean score : -trim_qual_type mean & -trim_qual_window 5 <br>
1) Trim 5' based on qual : -trim_qual_left 20 (Phred score) <br>
2) Trim 3' based on qual : -trim_qual_right 20 <br>
3) Discard reads shorter than 100 nts : -min_len 100 <br>
4) Filter sequence with mean qual score below 20 : -min_qual_mean 20 <br>
5) Discard low-complexity reads : -lc_method entropy & -lc_threshold 70 (ala Vlaminck) <br>
6) Remove exact duplicates : -derep 1 & -derep_min 2 <br>

#### 4.2.1) QC

Goal : Check if there is a notable improvement compared to QC done on raw reads.

In [ ]:
#### Code ###
fastqc='/home/erwann/Software/FastQC/Outputs';

In [ ]:
#### Code ###
# FastQC
mkdir -p $fastqc/TTV_qual \
         $out/QC;
cd $out;
for f in *.fastq.gz;
do fastqc --outdir $fastqc/TTV_qual/ -f fastq $f;
done &>> QC/fastqc_verbose.txt;

# MultiQC
cd $fastqc/TTV_qual;
mkdir -p P1 P2 P31 P32;
for f in p1_*; do mv $f P1; done;
for f in p2_*; do mv $f P2; done;
for f in p31_*; do mv $f P31; done;
for f in p32_*; do mv $f P32; done;
cd ~/Software/MultiQC/Outputs;
multiqc -i TTV_qual_p1 -n TTV_qual_p1 -v $fastqc/TTV_qual/P1;
multiqc -i TTV_qual_p2 -n TTV_qual_p2 -v $fastqc/TTV_qual/P2;
multiqc -i TTV_qual_p31 -n TTV_qual_p31 -v $fastqc/TTV_qual/P31;
multiqc -i TTV_qual_p32 -n TTV_qual_p32 -v $fastqc/TTV_qual/P32;
cp -r TTV_qual_* $out/QC;
cd $fastqc/TTV_qual;
mv P1/* P2/* P31/* P32/* .;
rm -r P1 P2 P31 P32;

In [ ]:
#### Code ###
cd $out/QC;
chromium-browser *.html;

#### 4.2.2) Merge FASTQ files belonging to the same sample

First, create a backup folder with unmerged FASTQ :

In [ ]:
#### Code ###
cd $out;
mkdir -p Before_sample_merging;
cp *.fastq.gz Before_sample_merging;

In [ ]:
#### Code ###
bash rename_to_sample.sh;
rm p*.*fastq.gz;

Ps : Content of rename_to_sample.sh :

In [ ]:
#### Code ###
cat p31_r74_qual.fastq.gz p32_r74_qual.fastq.gz > d1_qual.fastq.gz;
cat p31_r75_qual.fastq.gz p32_r75_qual.fastq.gz > d3_qual.fastq.gz;
mv p1_r2_qual.fastq.gz d4_qual.fastq.gz;
...
mv p2_r54_qual.fastq.gz r44_qual.fastq.gz;
mv p2_r55_qual.fastq.gz r45_qual.fastq.gz;
cat p2_r56_qual.fastq.gz p31_r33_qual.fastq.gz p32_r33_qual.fastq.gz \
    p31_r63_qual.fastq.gz p32_r63_qual.fastq.gz > r46_qual.fastq.gz;

Number of reads in quality filtered files :

In [ ]:
#### Code ###
for f in *fastq.gz;
do lines=$(zgrep -Ec "$" $f);
   printf $(($lines/4));
   printf '\t'$f'\n';
done;

### 4.3) Human reads removal

#### 4.3.1) Load data on genotoul :

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Output;
sudo scp -r Reads_qual escaon@genotoul.toulouse.inra.fr:/home/escaon/save/TTV;

We are using BWA-0.7.15 to map reads on the human genome.

#### 4.3.2) Build BWA index :

Content of bwa_idx.sh :

In [ ]:
#### Code ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=48G
#$ -l h_vmem=60G
#$ -N GRCh38_bwa_idx
#$ -o /home/escaon/work/TTV/bwa_idx_o.txt
#$ -e /home/escaon/work/TTV/bwa_idx_e.txt

BWA='/usr/local/bioinfo/src/bwa/bwa-0.7.15/bwa';
path='/home/escaon/save/TTV/Data';

$BWA index -p GRCh38_p11_bwtsw -a bwtsw $path/GCA_000001405.26_GRCh38.p11_genomic.fna;

#### 4.3.3) Map reads :

Content of bwa_mem.sh :

In [ ]:
#### Code ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=12G
#$ -N TTV_bwa_mem
#$ -o /home/escaon/work/TTV/ttv_bwa_mem_20170907_o.txt
#$ -e /home/escaon/work/TTV/ttv_bwa_mem_20170907_e.txt

# qsub -pe parallel_smp 8 bwa_mem.sh

BWA='/usr/local/bioinfo/src/bwa/bwa-0.7.15/bwa';
save='/home/escaon/save/TTV';
work='/home/escaon/work/TTV';
nt=8;

mkdir -p $work/Reads_qual_hm;
cd $save/Reads_qual;
for f in *.fastq.gz;
do $BWA mem -t $nt \
            -R '@RG\tID:'${f%%.*}'\tLB:'${f%%.*}'\tPL:IONTORRENT\tPU:GenoLim\tSM:'${f%%.*} \
            $work/Bwa_idx/GRCh38_p11_bwtsw \
            <(unpigz -c $f) |\
            samtools view -bh -f 4 \
            > $work/Reads_qual_hm/${f%%.*}.bam;
done;

In [ ]:
#### Code ###
grep -i 'error' ttv_bwa_mem_20170907_e.txt;

#### 4.3.4) Validate BAMs :

Content of picard_validate.sh :

In [ ]:
#### Code ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=32G
#$ -l h_vmem=40G
#$ -N TTV_picard_validate
#$ -o /home/escaon/work/TTV/ttv_picard_val_o.txt
#$ -e /home/escaon/work/TTV/ttv_picard_val_e.txt

work='/home/escaon/work/TTV';
module load bioinfo/Java8;
PICARD='/usr/local/bioinfo/src/picard-tools/picard-tools-2.1.1/picard.jar';

cd $work/Reads_qual_hm;
for f in *.bam;
do echo $f; java -jar $PICARD ValidateSamFile I=$f MODE=SUMMARY;
done;

In [ ]:
#### Code ###
grep -i 'error' ttv_picard_val_e.txt;
# & manual review of ttv_picard_val_o.txt

#### 4.3.5) Sort & index BAMs :

Content of samtools_sort_idx.sh :

In [ ]:
#### Code ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=40G
#$ -l h_vmem=48G
#$ -N TTV_samtools_sort_idx
#$ -o /home/escaon/work/TTV/ttv_samtools_sortidx_o.txt
#$ -e /home/escaon/work/TTV/ttv_samtools_sortidx_e.txt

work='/home/escaon/work/TTV';

cd $work/Reads_qual_hm;
for f in *.bam;
do samtools sort -o ${f%.*}"_sorted.bam" $f;
done;
for f in *_sorted.bam;
do samtools index $f;
done;

In [ ]:
#### Code ###
grep -i 'error' ttv_samtools_sortidx_e.txt;

#### 4.3.6) BAM to FASTA :

Content of samtools_fasta.sh :

In [ ]:
#### Code ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=32G
#$ -l h_vmem=40G
#$ -N TTV_samtools_fasta
#$ -o /home/escaon/work/TTV/ttv_samtools_fasta_o.txt
#$ -e /home/escaon/work/TTV/ttv_samtools_fasta_e.txt

work='/home/escaon/work/TTV';

cd $work/Reads_qual_hm;
for f in *_sorted.bam;
do samtools fasta $f > ${f%_*}.fasta;
done;

#### 4.3.7) DL these FASTA files locally :

In [ ]:
#### Code ###
sudo scp escaon@genotoul.toulouse.inra.fr:/home/escaon/work/TTV/Reads_qual_hm/*.fasta \
         /home/erwann/Desktop/TTV/BISCEm/Output/Reads_qual_hm;

Number of reads in filtered files : <br>
Ps : Could have been checked in BAM files with : samtools view -c toto.bam;

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Output/Reads_qual_hm;
for f in *.fasta;
do printf $f'\t';
   grep -c '^>' $f;
done;

### 4.4) 5' UTR reads removal

Rappel de l'objectif ici : Sachant que les génotypes de TTV partagent une zone en 5' qui est fortement conservée, il serait bon de ne pas tenir compte des reads qui s'alignent pour une majeur partie de leur longueur sur cette zone 5', pour éviter de créer des biais au niveau du compte des reads assignés à chaque génotype (en gros on préfère se focaliser sur les reads vraiment discriminants).

Ok, donc je pars sur ce consensus +++ conservé de 93nts. Cf : /home/erwann/Desktop/TTV/BISCEm/Data/TTV/Genomes_MSA/MultAlin result.html

Nb : En 3' de ce consensus, on retrouve directement pas mal de variabilité entre les génotypes. C'est moins vrai en 5', je dirais que les 67 nts en amont du consensus sont relativement conservés, donc pas forcément très discriminants pour nos objectifs.

Je pense donc appliquer la logique suivante :
Pour chaque jeu de données après filtre sur la qualité et soustraction des reads humain, aligner les reads restants contre le consensus de 93 nts.
Si un read possède un hit de 25 ou plus nts dans cette zone de 93nts alors :
Si ce read possède, en 3' du consensus, 50 ou plus nts non-impliqués dans le hit , on le conserve (car ces 50+ nts seront discriminants).
Si ce read possède, en 5' du consensus + 67 nts (pour tenir compte de la partie environ conservée), 50 ou plus nts non-impliqués dans le hit, on le conserve (car ces 50+ nts seront discriminants).
Si le read n'a pas assez de nts discriminants en 3' ou en 5' du consensus, alors il sera retiré du jeu de données.

Content of discard_5prime_consensus_reads.py :

In [ ]:
#### Code ###
#!/usr/bin/env python
import os
import sys

usage = '\t --------\n' \
        '\t| usage  : python discard_5prime_consensus_reads file_1\n' \
        '\t| input  : file_1 = blastn_vs_5prime_consensus.tsv\n' \
        '\t| output : a file containing read names to discard\n' \
        '\t --------'

if len(sys.argv) != 2:
    print(usage)
    sys.exit()

query_dic = {}
with open(sys.argv[1], 'r') as tsv:
    for row in tsv:
        columns = row.split('\t')
        qseqid, length, qstart, qend, sstart, send, qlen, sstrand = columns[0], int(columns[3]), int(
            columns[6]), int(columns[7]), int(columns[8]), int(columns[9]), int(columns[12]), columns[14].rstrip()
        if not qseqid in query_dic:
            if length >= 25:
                if sstrand == 'plus':
                    five_prime_overhead = (qstart - 1) - (sstart - 1) - 67
                    three_prime_overhead = (qlen - qend) - (93 - send)
                elif sstrand == 'minus':
                    five_prime_overhead = (qlen - qend) - (send - 1) - 67
                    three_prime_overhead = (qstart - 1) - (93 - sstart)
                else:
                    print('------------ERROR------------')
                if five_prime_overhead >= 50 or three_prime_overhead >= 50:
                    query_dic[qseqid] = 'keep'
                else:
                    query_dic[qseqid] = 'discard'

file = sys.argv[1].split('/')
sample = file[-1].split('.')[0]

with open(sample + '.discard', 'a') as out:
    for k, v in query_dic.items():
        if v == 'discard':
            out.write(k + "\n")

Apply this filtering step :

In [ ]:
#### Code ###
biscem='/home/erwann/Desktop/TTV/BISCEm';
blast='/home/erwann/Software/Ncbi_blast_2.6.0+/bin';
mkdir -p $biscem/Output/Reads_qual_hm_fivep;
cd $biscem/Output/Reads_qual_hm;
for f in *.fasta;
do $blast/blastn -subject $biscem/Data/TTV/ttv_5prime_consensus.fasta \
                 -query $f \
                 -reward 1 \
                 -penalty -3 \
                 -word_size 12 \
                 -gapopen 5 \
                 -gapextend 2 \
                 -evalue 0.0001 \
                 -dust "no" \
                 -perc_identity 90 \
                 -culling_limit 2 \
                 -out ${f%.*}.tsv \
                 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart \
                          qend sstart send evalue bitscore qlen nident sstrand";
                 python $biscem/Scripts/discard_5prime_consensus_reads.py ${f%.*}.tsv;
                 grep '^>' $f | sort > ${f%.*}.headers; sed -i 's/^>//g' ${f%.*}.headers;
                 sort ${f%.*}.discard -o ${f%.*}.discard;
                 comm -23 ${f%.*}.headers ${f%.*}.discard > ${f%.*}.keepers;
                 seqtk subseq $f ${f%.*}.keepers > $biscem/Output/Reads_qual_hm_fivep/$f;
done;
rm *.tsv *.headers *.discard *.keepers;

Number of reads in filtered files :

In [ ]:
#### Code ###
cd ~/Desktop/TTV/BISCEm/Output/Reads_qual_hm_fivep;
for f in *.fasta;
do printf $f'\t';
   grep -c '^>' $f;
done;

Upload them on genotoul & CALI :

In [ ]:
#### Code ####### Code ###
sudo scp -r /home/erwann/Desktop/TTV/BISCEm/Output/Reads_qual_hm_fivep \
            escaon@genotoul.toulouse.inra.fr:/home/escaon/save/TTV;
sudo scp -r /home/erwann/Desktop/TTV/BISCEm/Output/Reads_qual_hm_fivep \
            scaonp01@cali.unilim.fr://home/scaonp01/TTV;

## 5) TTV taxonomical analysis

It was decided to focus on Orf1 for this analysis, based on 65% homology (Maggi, etc...)

### 5.1) Retrieve Orf1 sequences for our 57 genotypes

- Manual inspection of the 57 genotypes entries for orf1 :

cd /home/erwann/Desktop/TTV/BISCEm/Data/TTV;
while read line;
do chromium-browser https://www.ncbi.nlm.nih.gov/nuccore/$line;
done < an_all_ttv_genotypes.txt;

cd /home/erwann/Desktop/TTV/BISCEm/Data/TTV;
while read line;
do touch ../../Orf1/$line.bed;
done < an_all_ttv_genotypes.txt;

- Then fill the 57 BED files manually based on webpage info (start pos -1 plz)
e.g. AB038621.1.bed => AB038621.1 665 2867
- Those have no annotated CDS for Orf1 :

AX025718.1
AX025830.1
AX174942.1
KP343839.1
NC_015783.1
- Go to https://www.ebi.ac.uk/Tools/psa/genewise/
& align this Orf1 protein sequence against them (https://www.ncbi.nlm.nih.gov/nuccore/NC_002076.2)
They all have a hit <=> orf1 => put this info in their BED.

- use seqtk to retrieve orf1 sequence only :

cd /home/erwann/Desktop/TTV/BISCEm/Data/TTV/All_ttv_genotypes_one_per_fasta;
for f in *.fasta;
do seqtk subseq $f ../Orf1/${f%.*}.bed > ../Orf1/${f%.*}.fasta;
done;

- Need a step which checks that results nts len is / by 3.

cd ../Orf1;
for f in *.fasta; do printf $f" : "; grep -v ">" $f | wc | awk '{print ($3-$1)/3}'; done;

### 5.2) Create taxo groups based on Orf1 homology

Voici les résultats de la phylogénie orf1 (alignement mafft+rooted UPGMA tree). <br>
En comptant 65% de similitude on part sur 30 groupes : cf fichiers pim et groupes-pim (percentage identity matrix)

## 6 Assign reads to microbial DBs & TTV genotypes

This task will be done using blast 2.6.0+

Ps : For fungal & bacterial blastn : <br>

- Let's define TSV output as classical outfmt 6 fields + : <br>
qlen # Query sequence length (needed to filter hits shorter than 90% of qlen) <br>
saccver # Subject accession.version (in case of future need for eutils) <br>
staxids # Subject Taxonomy ID(s) (If we want top X family / genus at some point) <br>
sskingdoms # Subject Super Kingdom(s) (might be enough (rather than eutils)) <br>
sblastnames # Subject Blast Name(s) (might be enough (rather than eutils)) <br>
sscinames # Subject Scientific Name(s) (might be enough (rather than eutils)) <br>


- Which translate into this argument : <br>
-outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen saccver staxids sskingdoms sblastnames sscinames" <br>


- But taxonomic fields & nt fields (saccver, staxids, sskingdoms, sblastnames, sscinames) are not working with custom DB. <br>

Ps : with NT & no staxids, we can still access tax infos with AN + efetch : <br>

In [ ]:
### Code ###
# Find staxids with accession number
# Result => AN   staxids
an='FJ070571';
an='XM_012200431';
efetch -db nuccore \
       -id $an \
       -format xml \
       | xtract -pattern Bioseq-set_seq-set \
                -division Seq-entry \
                -group Bioseq \
                -if Textseq-id_accession -equals $an \
                -element Textseq-id_accession \
                -branch Bioseq_descr \
                -block Org-ref_db \
                -element Object-id_id;

# Fuck sometimes taxon is in Bioseq-set_descr (XM_012200431)

Thus we're gonna stick with -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen"

Ps : For viral blastn : <br>

We need additional outfmt 6 fields to parse for Orf1 : <br>
qstart # Start of alignment in query <br>
qend # End of alignment in query <br>
slen # Subject sequence length <br>
sstart # Start of alignment in subject <br>
send # End of alignment in subject <br>
sstrand # Subject Strand <br>

Thus yielding : -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen qstart qend slen sstart send sstrand"

### 6.1) Create databases

#### 6.1.1) Viral DBs

In [ ]:
### CODE ###
# Initial rr_viral_plus_ttv

# rr_viral_completeG

### USELESS AT THE END OF THE DAY ###
# Add -100 / +100 to BED positions
cd ~/Desktop/TTV/BISCEm/Data/TTV/Orf1;
mkdir -p ../Orf1_extended_100nts;
for f in *.bed;
do awk -F "\t" '{print $1"\t"$2-100"\t"$3+100}' $f > ../Orf1_extended_100nts/$f;
done;

# Retrieve corresponding sequences with seqtk
cd ~/Desktop/TTV/BISCEm/Data/TTV/All_ttv_genotypes_one_per_fasta;
for f in *.fasta;
do seqtk subseq $f ../Orf1_extended_100nts/${f%.*}.bed > ../Orf1_extended_100nts/$f;
done;

Ps : Discuter et faire ici ajustement DB viral // TTV et aux seqs Orf1 et g_complet => OUI.

Ps : Génomes complets TTV non-circularisés "artificiellement" car créerait un biais.

#### 6.1.2) Fungal DB

#### 6.1.3) Bacterial DB

In [ ]:
### CODE ###
blast='/home/erwann/Software/Ncbi_blast_2.6.0+/bin';
unpigz hmp_bacterial.fasta.gz;
$blast/makeblastdb -in hmp_bacterial.fasta -input_type fasta -dbtype nucl -out hmp_bacterial;
pigz hmp_bacterial.fasta;

### 6.2) Blastn vs viral DBs

Done locally :

In [ ]:
### CODE ###
biscem='/home/erwann/Desktop/TTV/BISCEm';
blast='/home/erwann/Software/Ncbi_blast_2.6.0+/bin';
mkdir -p $biscem/Output/Viral_blastn;
nt=8;

cd $biscem/Output/Reads_qual_hm_fivep;
for f in *.fasta;
do $blast/blastn -db $biscem/Data/Blast_DB/rr_viral_plus_ttv \
                 -query $f \
                 -reward 1 \
                 -penalty -3 \
                 -word_size 12 \
                 -gapopen 5 \
                 -gapextend 2 \
                 -evalue 0.0001 \
                 -dust "no" \
                 -perc_identity 90 \
                 -culling_limit 2 \
                 -out $biscem/Output/Viral_blastn/${f%_*}"_vs_viral.tsv" \
                 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart \
                          qend sstart send evalue bitscore qlen slen sstrand" \
                 -num_threads $nt;
done;

### OLD -> TO DEL WHEN ABOVE RUNS ###
biscem='/home/erwann/Desktop/TTV/BISCEm';
blast='/home/erwann/Software/Ncbi_blast_2.6.0+/bin';
mkdir -p $biscem/Vlaminck_pipe/Viral_blastn;
nt=8;
c $biscem/Vlaminck_pipe/Reads_filtered_100_minusHm_minus5prime;
for f in *.fasta;
do $blast/blastn -db $biscem/Data/Viral/DB/rr_viral_plus_ttv -query $f -reward 1 -penalty -3 -word_size 12 \
-gapopen 5 -gapextend 2 -evalue 0.0001 -dust "no" -perc_identity 90 -culling_limit 2 \
-out $biscem/Vlaminck_pipe/Viral_blastn/${f%.*}"_vs_viral.tsv" \
-outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend \
sstart send evalue bitscore qlen nident sstrand" -num_threads $nt;
done;

### 6.3) Blastn vs fungal DB

Done on CALI :

Create an initial script :

In [ ]:
### CODE ###
#!/bin/bash
#SBATCH --partition=normal
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=12
#SBATCH --mem-per-cpu=8192
#SBATCH --time=1-23:59:59
#SBATCH --mail-user=erwan.scaon@unilim.fr
#SBATCH --mail-type=begin
#SBATCH --mail-type=end
#SBATCH --error=/home/scaonp01/scratch/TTV/ttv_blastn_fungal_d5.verbose
#SBATCH --output=/home/scaonp01/scratch/TTV/ttv_blastn_fungal_d5.verbose
#SBATCH --job-name=ttv_blastn_fungal_d5

# sbatch blastn_fungal_d5.sh

##############################
### Paths, tools & modules ###
##############################
nt=12;
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK;
ttv='/home/scaonp01/TTV';
scratch='/home/scaonp01/scratch/TTV';
blast='/home/scaonp01/Softwares/Ncbi_blast_plus_2.6.0';

################
### Commands ###
################
cd $ttv/Reads_qual_hm_fivep;
for f in d5_qual.fasta;
do $blast/blastn -db $ttv/DB/rr_fungi \
                 -query $f \
                 -reward 1 \
                 -penalty -3 \
                 -word_size 12 \
                 -gapopen 5 \
                 -gapextend 2 \
                 -evalue 0.0001 \
                 -dust "no" \
                 -perc_identity 90 \
                 -culling_limit 2 \
                 -out $scratch/${f%_*}"_vs_fungal.tsv" \
                 -outfmt "6 qseqid sseqid pident length mismatch gapopen \
                          qstart qend sstart send evalue bitscore qlen" \
                 -num_threads $nt;
done;

Launch script :

In [ ]:
### CODE ###
sbatch blastn_fungal_d5.sh;

Duplicate it for the other samples :

In [ ]:
### CODE ###
# Check what we are going to replace
for id in 'd5';
do grep --color -i $id blastn_fungal_"$id".sh;
done;

# Dup scripts
for id in 'd1' 'd3' 'd4' 'd8' 'd10' 'd11' 'd12' 'd13' 'd14' 'd15' 'd16' 'd17' \
          'd19' 'd20' 'r1' 'r2' 'r3' 'r4' 'r5' 'r6' 'r7' 'r8' 'r9' 'r10' 'r11' \
          'r12' 'r13' 'r14' 'r15' 'r16' 'r17' 'r18' 'r19' 'r20' 'r21' 'r22' 'r23' \
          'r24' 'r25' 'r26' 'r27' 'r28' 'r29' 'r30' 'r31' 'r32' 'r33' 'r34' 'r35' \
          'r36' 'r37' 'r38' 'r39' 'r40' 'r41' 'r42' 'r43' 'r44' 'r45' 'r46';
do cp blastn_fungal_d5.sh blastn_fungal_"$id".sh;
done;

# Replace with new E name
for id in 'd1' 'd3' 'd4' 'd8' 'd10' 'd11' 'd12' 'd13' 'd14' 'd15' 'd16' 'd17' \
          'd19' 'd20' 'r1' 'r2' 'r3' 'r4' 'r5' 'r6' 'r7' 'r8' 'r9' 'r10' 'r11' \
          'r12' 'r13' 'r14' 'r15' 'r16' 'r17' 'r18' 'r19' 'r20' 'r21' 'r22' 'r23' \
          'r24' 'r25' 'r26' 'r27' 'r28' 'r29' 'r30' 'r31' 'r32' 'r33' 'r34' 'r35' \
          'r36' 'r37' 'r38' 'r39' 'r40' 'r41' 'r42' 'r43' 'r44' 'r45' 'r46';
do sed -i "s/d5/$id/" blastn_fungal_"$id".sh;
done;

# Launch
for id in 'd1' 'd3' 'd4' 'd8' 'd10' 'd11' 'd12' 'd13' 'd14' 'd15' 'd16' 'd17' \
          'd19' 'd20' 'r1' 'r2' 'r3' 'r4' 'r5' 'r6' 'r7' 'r8' 'r9' 'r10' 'r11' \
          'r12' 'r13' 'r14' 'r15' 'r16' 'r17' 'r18' 'r19' 'r20' 'r21' 'r22' 'r23' \
          'r24' 'r25' 'r26' 'r27' 'r28' 'r29' 'r30' 'r31' 'r32' 'r33' 'r34' 'r35' \
          'r36' 'r37' 'r38' 'r39' 'r40' 'r41' 'r42' 'r43' 'r44' 'r45' 'r46';
do sbatch blastn_fungal_"$id".sh;
done;

### 6.4) Blastn vs bacterial DB

Done on Genotoul :

Create an initial script (blastn_bacterial_d5.sh) :

In [ ]:
### Code ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=10G
#$ -N TTV_blastn_bacterial_d5
#$ -o /home/escaon/work/TTV/ttv_blastn_bacterial_d5_o.txt
#$ -e /home/escaon/work/TTV/ttv_blastn_bacterial_d5_e.txt

# qsub -pe parallel_smp $nt script.sh

work='/home/escaon/work/TTV';
save='/home/escaon/save/TTV';
blast="$work/NCBI_BlastPlus_2.6.0/bin";
mkdir -p $work/Bacterial_blastn;
nt=8;

cd $save/Reads_qual_hm_fivep;
for f in d5_qual.fasta;
do $blast/blastn -db $work/Blast_db/hmp_bacterial \
                 -query $f \
                 -reward 1 \
                 -penalty -3 \
                 -word_size 12 \
                 -gapopen 5 \
                 -gapextend 2 \
                 -evalue 0.0001 \
                 -dust "no" \
                 -perc_identity 90 \
                 -culling_limit 2 \
                 -out $work/Bacterial_blastn/${f%_*}"_vs_bacterial.tsv" \
                 -outfmt "6 qseqid sseqid pident length mismatch gapopen \
                          qstart qend sstart send evalue bitscore qlen" \
                 -num_threads $nt;
done;

Launch it :

In [ ]:
### CODE ###
qsub -pe parallel_smp 8 blastn_bacterial_d5.sh;

Duplicate it for the other samples :

In [ ]:
#### Code ###
# Check what we are going to replace
for id in 'd5';
do grep --color -i $id blastn_bacterial_"$id".sh;
done;

# Dup scripts
for id in 'd1' 'd3' 'd4' 'd8' 'd10' 'd11' 'd12' 'd13' 'd14' 'd15' 'd16' 'd17' \
          'd19' 'd20' 'r1' 'r2' 'r3' 'r4' 'r5' 'r6' 'r7' 'r8' 'r9' 'r10' 'r11' \
          'r12' 'r13' 'r14' 'r15' 'r16' 'r17' 'r18' 'r19' 'r20' 'r21' 'r22' 'r23' \
          'r24' 'r25' 'r26' 'r27' 'r28' 'r29' 'r30' 'r31' 'r32' 'r33' 'r34' 'r35' \
          'r36' 'r37' 'r38' 'r39' 'r40' 'r41' 'r42' 'r43' 'r44' 'r45' 'r46';
do cp blastn_bacterial_d5.sh blastn_bacterial_"$id".sh;
done;

# Replace with new E name
for id in 'd1' 'd3' 'd4' 'd8' 'd10' 'd11' 'd12' 'd13' 'd14' 'd15' 'd16' 'd17' \
          'd19' 'd20' 'r1' 'r2' 'r3' 'r4' 'r5' 'r6' 'r7' 'r8' 'r9' 'r10' 'r11' \
          'r12' 'r13' 'r14' 'r15' 'r16' 'r17' 'r18' 'r19' 'r20' 'r21' 'r22' 'r23' \
          'r24' 'r25' 'r26' 'r27' 'r28' 'r29' 'r30' 'r31' 'r32' 'r33' 'r34' 'r35' \
          'r36' 'r37' 'r38' 'r39' 'r40' 'r41' 'r42' 'r43' 'r44' 'r45' 'r46';
do sed -i "s/d5/$id/" blastn_bacterial_"$id".sh;
done;

# Launch
for id in 'd1' 'd3' 'd4' 'd8' 'd10' 'd11' 'd12' 'd13' 'd14' 'd15' 'd16' 'd17' \
          'd19' 'd20' 'r1' 'r2' 'r3' 'r4' 'r5' 'r6' 'r7' 'r8' 'r9' 'r10' 'r11' \
          'r12' 'r13' 'r14' 'r15' 'r16' 'r17' 'r18' 'r19' 'r20' 'r21' 'r22' 'r23' \
          'r24' 'r25' 'r26' 'r27' 'r28' 'r29' 'r30' 'r31' 'r32' 'r33' 'r34' 'r35' \
          'r36' 'r37' 'r38' 'r39' 'r40' 'r41' 'r42' 'r43' 'r44' 'r45' 'r46';
do qsub -pe parallel_smp 8 blastn_bacterial_"$id".sh;
done;

### 6.5) Hit count per TTV genotypes